<a href="https://colab.research.google.com/github/selfhiam/AI-Chatbot/blob/main/%ED%95%99%EC%8A%B5test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytorch_lightning

In [ ]:
import numpy as np
import pandas as pd
import torch
# from pytorch_lightning import Trainer
# from pytorch_lightning.callbacks import ModelCheckpoint
# from pytorch_lightning.core.lightning import LightningModule
from torch.utils.data import DataLoader, Dataset
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
Q_TKN = "<usr>"
A_TKN = "<sys>"
L_TKN = "<label>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'


In [ ]:
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK)
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

In [ ]:
Chatbot_Data = pd.read_csv("/content/drive/MyDrive/test/test4.csv", names=['1', 'Q', 'A'])
Chatbot_Data

,1,Q,A
0,0,오랜만에 할머니를 만나서 기분이 어때?,진작에 할머니 안 만나고 뭐했어요!
1,1,오랜만에 할머니를 만나서 기분이 어때?,오랜만인지도 몰랐네요~.
2,2,오랜만에 할머니를 만나서 기분이 어때?,왜 할머니를 이제서야 만나는거에요!
3,3,오랜만에 할머니를 만나서 기분이 어때?,어~ 좋은 거 같아요 ㅎㅎ~
4,4,오랜만에 할머니를 만나서 기분이 어때?,너무 좋아요! 맛있는 거 해주시겠죠?
...,...,...,...
7995,5,챗쪽아 숙제는 다 했니?,네... 근데 아직 다 안 한 것 같아서 불안해요.
7996,6,챗쪽아 숙제는 다 했니?,"아뇨, 그래도 한쪽 남았어요."
7997,7,챗쪽아 숙제는 다 했니?,"헉, 숙제를 하는거 까먹었어요 죄송해요!"
7998,8,챗쪽아 숙제는 다 했니?,네! 저 이제 잠깐 친구랑 놀다 올게요!


In [ ]:
# 챗봇 데이터를 처리하는 클래스를 만든다.
class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=50):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        self.max_len = max_len
        self.q_token = Q_TKN
        self.a_token = A_TKN
        self.l_token = L_TKN
        self.sent_token = SENT
        self.eos = EOS
        self.mask = MASK
        self.tokenizer = koGPT2_TOKENIZER

    def __len__(self):  # chatbotdata 의 길이를 리턴한다.
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        turn = self._data.iloc[idx]
        l = str(turn["1"])

        q = turn["Q"]  # 질문을 가져온다.
        q = re.sub(r"([?.!,~])", r" ", q)  # 구둣점들을 제거한다.
        q = re.sub(r'([ㄱ-ㅎㅏ-ㅣ])\1+', r'\1', q) # 자음이나 모음만 연속되는 것 한개만 남기고 삭제


        a = turn["A"]  # 답변을 가져온다.
        a = re.sub(r"([?.!,])", r" ", a)  # 구둣점들을 제거한다.
        a = re.sub(r'([ㄱ-ㅎㅏ-ㅣ])\1+', r'\1', a) # 자음이나 모음만 연속되는 것 한개만 남기고 삭제

        l_toked = self.tokenizer.tokenize(self.l_token + l + self.sent_token)
        l_len = len(l_toked)

        q_toked = self.tokenizer.tokenize(self.q_token + q + self.sent_token)
        q_len = len(q_toked)

        t_toked = (l_toked + q_toked)
        t_len = len(t_toked)

        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_toked)


        #질문의 길이가 최대길이보다 크면
        if t_len > self.max_len:
            a_len = self.max_len - t_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                t_toked = t_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로
                t_len = len(t_toked)
                a_len = self.max_len - t_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        #질문의 길이 + 답변의 길이가 최대길이보다 크면
        if t_len + a_len > self.max_len:
            a_len = self.max_len - t_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                t_toked = t_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로
                t_len = len(t_toked)
                a_len = self.max_len - t_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)


        # 답변 labels = [mask, mask, ...., mask, ..., <bos>,..답변.. <eos>, <pad>....]
        labels = [self.mask,] * t_len + a_toked[1:]

        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
        mask = [0] * t_len + [1] * a_len + [0] * (self.max_len - t_len - a_len)
        # 답변 labels을 index 로 만든다.
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        # 최대길이만큼 PADDING
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        # 질문 + 답변을 index 로 만든다.
        token_ids = self.tokenizer.convert_tokens_to_ids(l_toked + q_toked + a_toked)
        # 최대길이만큼 PADDING
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        #질문+답변, 마스크, 답변
        return (token_ids, np.array(mask), labels_ids)


In [ ]:
def collate_batch(batch):
    data = [item[0] for item in batch]
    mask = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.LongTensor(data).to(device), torch.LongTensor(mask).to(device), torch.LongTensor(label).to(device)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_set = ChatbotDataset(Chatbot_Data, max_len=40)
#윈도우 환경에서 num_workers 는 무조건 0으로 지정, 리눅스에서는 2
train_dataloader = DataLoader(train_set, batch_size=32, num_workers=0, shuffle=True, collate_fn=collate_batch)


In [ ]:
model.to(device)
model.train()


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [ ]:
learning_rate = 3e-5
criterion = torch.nn.CrossEntropyLoss(reduction="none")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epochs = 30
Sneg = -1e18

In [ ]:
# for batch_idx, samples in enumerate(train_dataloader):
#     token_ids, mask, label = samples
#     print(len(token_ids), len(mask), len(label))

<ipython-input-13-d9c5350f11d2>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  return torch.LongTensor(data).to(device), torch.LongTensor(mask).to(device), torch.LongTensor(label).to(device)


32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
32 32 32
3

In [ ]:
print ("start")
best_loss = float('inf')  # 양의 무한대로 초기화
modelname = "GPT2_skt_kogpt2-base-v2"
for epoch in range(epochs):
    for batch_idx, samples in enumerate(train_dataloader):
        optimizer.zero_grad()
        token_ids, mask, label = samples
        out = model(token_ids)
        out = out.logits
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
        mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
        loss = criterion(mask_out.transpose(2, 1), label)
        # 평균 loss 만들기 avg_loss[0] / avg_loss[1] <- loss 정규화
        avg_loss = loss.sum() / mask.sum()
        avg_loss.backward()
        # 학습 끝
        optimizer.step()
    print(avg_loss)
    # 새로운 최고 손실이면 모델 저장
    if avg_loss < best_loss:
          best_loss = avg_loss
          torch.save({'model_state_dict': model.state_dict(),'optimizer_state_dict': optimizer.state_dict()}, f'{modelname}-{device}.pth')
          print(f'새로 모델이 저장되었습니다. 저장명 : {modelname}-{device}.pth LOSS : {best_loss}')
print ("end")


<ipython-input-8-d9c5350f11d2>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  return torch.LongTensor(data).to(device), torch.LongTensor(mask).to(device), torch.LongTensor(label).to(device)
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


start
tensor(23.5805, device='cuda:0', grad_fn=<DivBackward0>)
새로 모델이 저장되었습니다. 저장명 : GPT2_skt_kogpt2-base-v2-cuda.pth LOSS : 23.580528259277344
tensor(23.3079, device='cuda:0', grad_fn=<DivBackward0>)
새로 모델이 저장되었습니다. 저장명 : GPT2_skt_kogpt2-base-v2-cuda.pth LOSS : 23.307905197143555
tensor(25.6608, device='cuda:0', grad_fn=<DivBackward0>)
tensor(24.6880, device='cuda:0', grad_fn=<DivBackward0>)
tensor(22.7843, device='cuda:0', grad_fn=<DivBackward0>)
새로 모델이 저장되었습니다. 저장명 : GPT2_skt_kogpt2-base-v2-cuda.pth LOSS : 22.784326553344727
tensor(22.6654, device='cuda:0', grad_fn=<DivBackward0>)
새로 모델이 저장되었습니다. 저장명 : GPT2_skt_kogpt2-base-v2-cuda.pth LOSS : 22.66535186767578
tensor(23.2693, device='cuda:0', grad_fn=<DivBackward0>)
tensor(23.0452, device='cuda:0', grad_fn=<DivBackward0>)
tensor(24.3838, device='cuda:0', grad_fn=<DivBackward0>)
tensor(23.9163, device='cuda:0', grad_fn=<DivBackward0>)
tensor(23.4276, device='cuda:0', grad_fn=<DivBackward0>)
tensor(23.7446, device='cuda:0', grad_fn=<Di

In [ ]:
torch.save(model.state_dict(), 'Chatjjock_21_48.pth')

In [ ]:
import re
with torch.no_grad():
    while 1:
        t = input("user > ").strip()
        l = t[0]
        q = t[1:]
        if q == "quit":
            break
        a = ""
        while 1:
            input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(L_TKN + l + SENT + Q_TKN + q + SENT + 'sent' + A_TKN + a)).unsqueeze(dim=0).to(device)
            pred = model(input_ids)
            pred = pred.logits
            gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().cpu().numpy().tolist())[-1]
            if gen == EOS:
                break
            a += gen.replace("▁", " ")
        a = a.strip()
        if l == "9":
          a = re.sub(r"\s{2,}", "....", a)
        if l == "4":
          a = re.sub(r"\s{2,}", "!", a)
        print("Chatbot > {}".format(a))

user > 4학교 잘 다녀왔니?
Chatbot > 네!엄빠도 같이 다녀왔어요
user > 4점심은 뭐 먹었니?
Chatbot > 네!뭐든 다 잘 먹어요~
user > 4친구들이랑 잘 놀았니?
Chatbot > 네!친구들이랑 노는 건 항상 재밌었어요
user > 4놀러갈까?
Chatbot > 엄빠랑 같이 놀러 가요
user > 4저녁에 맛있는거 먹자!
Chatbot > 네!뭐 해 먹어요
user > 3quit
